In [1]:
import urllib
import bs4
import re
import pandas as pd
import random

from time import sleep

#help(bs4)

In [143]:
url_landing = "https://bwfbadminton.com/rankings/2/bwf-world-rankings/8/men-s-doubles/2020/12/?rows=100&page_no=1"
req = urllib.request.Request(url_landing, headers={'User-Agent':'Mozilla/5.0'})
webpage = urllib.request.urlopen(req).read()
landing = bs4.BeautifulSoup(webpage, "lxml")

In [185]:
def get_df(url):
    
    url_landing = url
    req = urllib.request.Request(url_landing, headers={'User-Agent':'Mozilla/5.0'})
    webpage = urllib.request.urlopen(req).read()
    landing = bs4.BeautifulSoup(webpage, "lxml")  
    
    rank = [x for x in range(1,101)]
    
    winrate = []
    prize = []
    temp_list = []

    for item in landing.find_all("td",{"align":"center"}):
        temp_list.append(item.get_text().split())

    temp_list = [x for x in temp_list if x!=[]]
    winrate = [x for x in temp_list if '-' in x]
    prize = [x for x in temp_list if x[0].startswith('$')]
    
    countries = []
    for item in landing.find_all("div",{"class":"country"}):
        countries.append(item.get_text().split()[0])
    if len(countries) == 200:
        del countries[1:200:2]
        
    players = []
    for item in landing.find_all("div",{"class":"player"}):
        players.append(item.get_text())
        
    change = []
    for item in landing.find_all("span",{"class":"ranking-change"}):
        change.append(int(item.get_text()))
    
    points = []
    for item in landing.find_all("td",{"class":"point"}):
        points.append(item.get_text().split())
    points = [x for x in points if x!=[]]
    
    data = pd.DataFrame({'RANK':rank,
                     'COUNTRY':countries,
                     'PLAYER':players,
                     'RANK_CHANGE':change,
                    'WINRATE':winrate,
                    'CAREER_PRIZE_MONEY':prize,
                    'POINTS_FOR_WORLD_TOUR':points})
    
    data.set_index('RANK', inplace = True)
    
    data['PLAYER'] = data['PLAYER'].str.lstrip('\n\n\n ').str.rstrip(' \n\n')
    
    wins = []
    losses = []
    for x in data['WINRATE'].values:
        wins.append(int(x[0]))
        losses.append(int(x[2]))

    data['WINS'] = wins
    data['LOSSES'] = losses
    data['WINRATE'] = data['WINS']/(data['WINS']+data['LOSSES'])*100
    
    data['CAREER_PRIZE_MONEY'] = [int(x[0].strip('$').replace(',','')[:-3]) for x in data['CAREER_PRIZE_MONEY']]
    
    data['TOURNAMENTS_PLAYED'] = [int(x[2]) for x in data['POINTS_FOR_WORLD_TOUR']]
    
    data['POINTS_FOR_WORLD_TOUR'] = [int(x[0].replace(',','')) for x in data['POINTS_FOR_WORLD_TOUR']]
    
    data['PLAYER'] = data['PLAYER'].str.replace('\n\n\n', ' / ')
    
    sleep(random.randint(3, 7))
    
    return data

In [186]:
men_singles = get_df("https://bwfbadminton.com/rankings/2/bwf-world-rankings/6/men-s-singles/2020/12/?rows=100&page_no=1")
women_singles =  get_df("https://bwfbadminton.com/rankings/2/bwf-world-rankings/7/women-s-singles/2020/12/?rows=100&page_no=1")
men_doubles = get_df("https://bwfbadminton.com/rankings/2/bwf-world-rankings/8/men-s-doubles/2020/12/?rows=100&page_no=1")
women_doubles = get_df("https://bwfbadminton.com/rankings/2/bwf-world-rankings/9/women-s-doubles/2020/12/?rows=100&page_no=1")
mixed_doubles = get_df("https://bwfbadminton.com/rankings/2/bwf-world-rankings/10/mixed-doubles/2020/12/?rows=100&page_no=1")

In [189]:
men_singles.to_csv('men_singles.csv',index = True)
women_singles.to_csv('women_singles.csv',index = True)
men_doubles.to_csv('men_doubles.csv',index = True)
women_doubles.to_csv('women_doubles.csv',index = True)
mixed_doubles.to_csv('mixed_doubles.csv',index = True)